In [1]:
import os

# Restrict PyTorch to only see GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch

if torch.cuda.is_available():
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available, using CPU.")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Number of GPUs available: 1
GPU 0: NVIDIA L40S
Using device: cuda:0


In [2]:
from spatialvla.datasets import RLDSBatchTransform, RLDSDataset

2024-11-06 17:19:32.123506: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 17:19:32.153052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 17:19:32.153097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 17:19:32.153997: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 17:19:32.159286: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from spatialvla.mobilevlm.model.mobilevlm import load_pretrained_vlm_for_vla
from scripts.spatialvla_config import ModelArguments, TrainingArguments
import transformers

model_args = ModelArguments()

tokenizer, model, image_processor, _ = load_pretrained_vlm_for_vla(
    model_args,
    load_8bit=False, 
    load_4bit=False,
    device='cuda',
)

[2024-11-06 17:19:37,696] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jellyho/miniconda3/envs/mobilevlm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using a model of type mobilevlm to instantiate a model of type spatialvla. This is not supported for all configurations of models and can yield errors.
Some weights of SpatialVLAForCausalLM were not initialized from the model checkpoint at remyxai/SpaceLLaVA-lite and are newly initialized: ['action_head.map_head.attention.out_proj.bias', 'action_head.map_head.attention.out_proj.weight', 'action_head.map_head.layer_norm.bias', 'action_head.map_head.mlp_block.fc2.bias', 'action_head.map_head.layer_norm.weight', 'action_head.map_head.p', 'action_head.map_head.attention.in_proj_bias', 'action_head.map_head.mlp_block.fc1.bias', 'action_head.map_head.mlp_block.fc2.weight', 'ac

tensor(False, device='cuda:0')


In [4]:
## RLDS Dataset loading
batch_transform = RLDSBatchTransform(
        tokenizer,
        image_processor,
    )
# Init complete

In [5]:
cfg = TrainingArguments()

In [6]:
vla_dataset = RLDSDataset(
        data_root_dir=cfg.data_root_dir,
        data_mix='lg_mix',
        batch_transform=batch_transform,
        shuffle_buffer_size=100,
        window_size=1,
        future_action_window_size=0
    )

2024-11-06 17:20:17.336853: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-06 17:20:17.740781: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-06 17:20:18.391216: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-06 17:20:18.493089: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization



######################################################################################
# Loading the following 2 datasets (incl. sampling weight):                         #
# lg_cup_color_rightarm: ===================================================0.424252 #
# lg_stack_cup_5hz: ========================================================0.575748 #
######################################################################################

Threads per Dataset: %s [1 1]
Reads per Dataset: %s [1 1]
Constructing datasets...


2024-11-06 17:20:18.619690: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization
2024-11-06 17:20:19.130774: I tensorflow/core/grappler/optimizers/data/replicate_on_split.cc:32] Running replicate on split optimization


Applying frame transforms on dataset...


In [7]:
from spatialvla.datasets.rlds.utils.data_utils import PaddedCollatorForActionPrediction

collator = PaddedCollatorForActionPrediction(tokenizer.model_max_length, tokenizer.pad_token_id, padding_side='right')

In [8]:

from torch.utils.data import DataLoader
dataloader = DataLoader(
        vla_dataset,
        batch_size=2,
        sampler=None,
        collate_fn=collator,
        num_workers=0,  # Important =>> Set to 0 if using RLDS; TFDS rolls its own parallelism!
    )

In [9]:
import torch
device_id = 0
for batch in dataloader:
    with torch.autocast('cuda', dtype=torch.float16):
        action = model.forward(
            input_ids=batch['input_ids'].to(device_id),
            images=batch['pixel_values'].to(device_id),
            attention_mask=batch['attention_mask'].to(device_id)
        )
    break

/home/jellyho/Bimanual_Imitation/MobileVLM-VLA/spatialvla/datasets/datasets.py:49: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  dataset_name, action = rlds_batch["dataset_name"], torch.Tensor(rlds_batch["action"]).to(torch.float16)


Input x shape: torch.Size([2, 202, 2048])
p shape: torch.Size([1, 1, 2048])
torch.Size([2, 1, 2048]) 0
After attention: out shape: torch.Size([2, 1, 2048])
tensor([[[ 0.9770, -0.4976, -0.6277,  ...,  0.0979, -1.3399,  0.9625]],

        [[ 0.9779, -0.6012, -0.5796,  ...,  0.0086, -1.3354,  1.1075]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)
tensor([[[ 2.8926, -1.2119, -3.0527,  ...,  2.2441, -0.9912,  0.8364]],

        [[ 2.9180, -1.4316, -3.0703,  ...,  2.0898, -0.9355,  1.2285]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>)


In [10]:
action

tensor([[[ 1.8740, -0.9097, -0.7334, -2.3711, -0.3816, -0.3276, -5.6367]],

        [[ 1.7061, -0.8086, -0.6655, -2.3457, -0.3774, -0.3972, -5.6836]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<ReshapeAliasBackward0>)